In [1]:
#Importing libraries and defining variables
import mediapipe as mp 
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2 as cv
import time

ModelPath = '/home/nkminion/Desktop/Python/HandGesture/hand_landmarker.task'
BaseOptions = mp.tasks.BaseOptions
HandLandMarker = mp.tasks.vision.HandLandmarker
HandLandMarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandMarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
Cam = cv.VideoCapture(0)
Timestamp = 0
Increment = 10
WindowName = 'OutputWindow'

2025-02-03 21:58:57.308415: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 21:58:57.344257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738600137.383832    5672 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738600137.395504    5672 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 21:58:57.433552: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
#Landmarker stuff
class LandmarkerAndResult():
	def __init__(self):
		self.result = vision.HandLandmarkerResult
		self.landmarker = vision.HandLandmarker
		self.createLandmarker()

	def createLandmarker(self):
		#Callback function
		def PrintRes(result: vision.HandLandmarkerResult , output_image: mp.Image , timestamp_ms: int):
			self.result = result

		options = vision.HandLandmarkerOptions(
			base_options = mp.tasks.BaseOptions(model_asset_path=ModelPath),
			running_mode = VisionRunningMode.LIVE_STREAM,
			num_hands = 1,
			min_hand_detection_confidence = 0.3,
			min_hand_presence_confidence = 0.3,
			min_tracking_confidence = 0.3,
			result_callback = PrintRes
		)

		self.landmarker = self.landmarker.create_from_options(options)

	def detect_async(self , frame):
		input = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
		self.landmarker.detect_async(image = input, timestamp_ms = int(time.time() * 1000))

	def close(self):
		self.landmarker.close()

In [3]:
def DrawLandmarks(image, DetectionResult: mp.tasks.vision.HandLandmarkerResult):
   try:
      if DetectionResult.hand_landmarks == []: # Empty
         return image
      else:
         HandLandmarksList = DetectionResult.hand_landmarks
         AnnotatedImage = np.copy(image)

         # Loop through the detected hands to visualize.
         for i in range(len(HandLandmarksList)):
            HandLandmarks = HandLandmarksList[i]
            
            # Draw the hand landmarks.
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
               landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in HandLandmarks])
            mp.solutions.drawing_utils.draw_landmarks(
               AnnotatedImage,
               hand_landmarks_proto,
               mp.solutions.hands.HAND_CONNECTIONS,
               mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
               mp.solutions.drawing_styles.get_default_hand_connections_style())
         return AnnotatedImage
   except:
      return image

In [4]:
def FingersRaised(image , DetectionResult: mp.tasks.vision.HandLandmarkerResult):
	try:
		# Data
		HandLandMarksList = DetectionResult.hand_landmarks

		Raised = [False , False , False , False , False]
		NumRaised = 0

		for i in range(len(HandLandMarksList)):
			HandLandmarks = HandLandMarksList[i]

			# All fingers excluding thumb
			for j in range (8,21,4):
				TipY = HandLandmarks[j].y
				DipY = HandLandmarks[j-1].y
				PipY = HandLandmarks[j-2].y
				McpY = HandLandmarks[j-3].y
				if TipY < min(DipY,PipY,McpY):
					Raised[int((j/4)-1)] = True
					NumRaised += 1
			#Thumb
			TipX = HandLandmarks[4].x
			DipX = HandLandmarks[3].x 
			McpX = HandLandmarks[1].x 
			PipX = HandLandmarks[2].x
			PalmX =HandLandmarks[0].x 

			#Left
			if (TipX < PalmX):
				if (TipX < min(DipX,PipX,McpX)):
					Raised[0] = True
					NumRaised += 1
			else:
				if (TipX > max(DipX,PipX,McpX)):
					Raised[0] = True
					NumRaised += 1
			
			print("FingerNum: "+str(NumRaised))
			for p in range(5):
				print("Finger "+str(p)+" : "+str(Raised[p]))
	except:
		print("Error")

In [5]:
#Open Camera and capture livefeed
def main():
	landmarker = LandmarkerAndResult()
	while Cam.isOpened():
		ret , frame = Cam.read()
		landmarker.detect_async(frame)
		frame = DrawLandmarks(frame,landmarker.result)
		FingersRaised(frame , landmarker.result)
		cv.imshow(WindowName , frame)
		c = cv.waitKey(5)
		if (c == 27):
			break
	landmarker.close()
	Cam.release()
	cv.destroyAllWindows()

if __name__ == '__main__':
	main()

I0000 00:00:1738600142.029777    5672 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1738600142.040837    5721 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1738600142.131798    5725 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738600142.176256    5723 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error


W0000 00:00:1738600142.671649    5723 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Index should be 1.0
Index should be 2.0
Index should be 3.0
Index should be 4.0
FingerNum: 5
Finger 0 : True
Finger 1 : True
Finger 2 : True
Finger 3 : True
Finger 4 : True
Index should be 1.0
Index should be 2.0
Index should be 3.0
Index should be 4.0
FingerNum: 5
Finger 0 : True
Finger 1 : True
Finger 2 : True
Finger 3 : True
Finger 4 : True
Index should be 1.0
Index should be 2.0
Index should be 3.0
Index should be 4.0
FingerNum: 5
Finger 0 : True
Finger 1 : True
Finger 2 : True
Finger 3 : True
Finger 4 : True
Index should be 1.0
Index should be 2.0
Index should be 3.0
Index should be 4.0
FingerNum: 5
Finger 0 : True
Finger 1 : True
Finger 2 : True
Finger 3 : True
Finger 4 : True
Index should be 1.0
Index should be 2.0
Index should be 3.0
Index should be 4.0
FingerNum: 5
Finger 0 : True
Finger 1 : True
Finger 2 : True
Finger 3 : True
Finger 4 : True
Index should be 1.0
Index should be 2.0
Index should be 3.0
Index should be 4.0
FingerNum: 5
Finger 0 : True
Finger 1 : True
Finger 2 :